## Deliverable 2. Create a Customer Travel Destinations Map.

In [9]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

#config_file = os.path.join("Weather_Database", "config")
# Import API key

from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Badvel,IN,14.7500,79.0500,83.25,72,100,2.98,overcast clouds
1,1,Souillac,MU,-20.5167,57.5167,74.35,68,40,18.41,scattered clouds
2,2,Olga,RU,43.7444,135.2892,29.80,63,88,9.31,overcast clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,33.46,75,75,34.52,heavy snow
4,4,Vila Velha,BR,-20.3297,-40.2925,80.55,78,20,18.41,light rain


In [12]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Badvel,IN,14.7500,79.0500,83.25,72,100,2.98,overcast clouds
1,1,Souillac,MU,-20.5167,57.5167,74.35,68,40,18.41,scattered clouds
4,4,Vila Velha,BR,-20.3297,-40.2925,80.55,78,20,18.41,light rain
5,5,Yenagoa,NG,4.9247,6.2642,77.38,91,22,5.21,few clouds
6,6,Manokwari,ID,-0.8667,134.0833,78.49,80,100,3.53,light rain
7,7,Avarua,CK,-21.2078,-159.7750,78.85,78,75,11.50,broken clouds
9,9,Rikitea,PF,-23.1203,-134.9692,75.00,79,26,9.62,scattered clouds
12,12,Acarau,BR,-2.8856,-40.1200,84.61,65,100,21.18,overcast clouds
15,15,Arraial Do Cabo,BR,-22.9661,-42.0278,76.96,83,75,6.91,broken clouds
16,16,Pozo Colorado,PY,-23.4923,-58.7972,73.40,59,99,5.70,overcast clouds


In [14]:
# 4a. Determine if there are any empty rows.
#preferred_cities_df.isnull().sum()
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 281 entries, 0 to 701
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              281 non-null    int64  
 1   City                 281 non-null    object 
 2   Country              281 non-null    object 
 3   Lat                  281 non-null    float64
 4   Lng                  281 non-null    float64
 5   Max Temp             281 non-null    float64
 6   Humidity             281 non-null    int64  
 7   Cloudiness           281 non-null    int64  
 8   Wind Speed           281 non-null    float64
 9   Current Description  281 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 24.1+ KB


In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
pref_cities_df = preferred_cities_df.dropna()

In [27]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = pref_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Badvel,IN,83.25,overcast clouds,14.7500,79.0500,
1,Souillac,MU,74.35,scattered clouds,-20.5167,57.5167,
4,Vila Velha,BR,80.55,light rain,-20.3297,-40.2925,
5,Yenagoa,NG,77.38,few clouds,4.9247,6.2642,
6,Manokwari,ID,78.49,light rain,-0.8667,134.0833,
7,Avarua,CK,78.85,broken clouds,-21.2078,-159.7750,
9,Rikitea,PF,75.00,scattered clouds,-23.1203,-134.9692,
12,Acarau,BR,84.61,overcast clouds,-2.8856,-40.1200,
15,Arraial Do Cabo,BR,76.96,broken clouds,-22.9661,-42.0278,
16,Pozo Colorado,PY,73.40,overcast clouds,-23.4923,-58.7972,


In [30]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame

    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.

    try:
         hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
   

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [33]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace('', np.nan, inplace=True)
hotel_df.tail(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
676,Mandal,IN,75.83,overcast clouds,25.4500,74.5667,पीपी वाले हनुमान जी
677,Quatre Cocos,MU,74.08,clear sky,-20.2078,57.7625,Sunrise Attitude Hotel
679,Thinadhoo,MV,83.23,overcast clouds,0.5333,72.9333,NaN
681,Itacare,BR,82.29,broken clouds,-14.2775,-38.9967,Ecoporan Hotel Charme Spa & Eventos
691,Nova Granada,BR,86.25,broken clouds,-20.5339,-49.3142,Casa do maikão
694,Dourbali,TD,76.46,heavy intensity rain,11.8091,15.8632,NaN
696,Luau,AO,72.68,broken clouds,-10.7073,22.2247,Casa- Bernardo Cândido
697,Grafton,US,74.95,few clouds,42.2070,-71.6856,The Grafton Inn
700,Kimbe,PG,76.17,light rain,-5.5502,150.1429,Kimbe Bay Hotel
701,Libertador General San Martin,AR,71.98,few clouds,-23.8064,-64.7876,Posada del Sol


In [34]:
hotel_df_new = hotel_df.dropna()
hotel_df_new.tail(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
672,Makakilo City,US,80.92,scattered clouds,21.3469,-158.0858,Marriott's Ko Olina Beach Club
673,Wanning,CN,75.81,overcast clouds,18.8003,110.3967,Paradise Hotel
676,Mandal,IN,75.83,overcast clouds,25.4500,74.5667,पीपी वाले हनुमान जी
677,Quatre Cocos,MU,74.08,clear sky,-20.2078,57.7625,Sunrise Attitude Hotel
681,Itacare,BR,82.29,broken clouds,-14.2775,-38.9967,Ecoporan Hotel Charme Spa & Eventos
691,Nova Granada,BR,86.25,broken clouds,-20.5339,-49.3142,Casa do maikão
696,Luau,AO,72.68,broken clouds,-10.7073,22.2247,Casa- Bernardo Cândido
697,Grafton,US,74.95,few clouds,42.2070,-71.6856,The Grafton Inn
700,Kimbe,PG,76.17,light rain,-5.5502,150.1429,Kimbe Bay Hotel
701,Libertador General San Martin,AR,71.98,few clouds,-23.8064,-64.7876,Posada del Sol


In [35]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df_new.to_csv(output_data_file, index_label="City_ID")

In [36]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} &#176;F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_new.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df_new[["Lat", "Lng"]]

In [37]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))